In [1]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

# from feature_descriptors.sift_descriptor import get_keypoint_and_descriptor
# from feature_matching.flann_matcher import flann_matcher
# from load_h5py_files import load_sift_output, load_flann_output, load_flann_from_images_name
from clustering.cluster_images import build_graph, graph_clustering
from batch_descriptor import batch_feature_descriptor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import os
import h5py
import numpy as np
import cv2 as cv

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
from torchvision.transforms.functional import pad

class PadToSize():
    def __init__(self, size, fill=0):
        self.size = size
        self.fill = fill

    def __call__(self, img):
        w, h = img.size
        pad_h = max(0, self.size - h)
        pad_w = max(0, self.size - w)
        padding = [pad_w//2, pad_h//2, pad_w - pad_w//2, pad_h - pad_h//2]
        return pad(img, padding, fill=self.fill)

        
'''Original dataset, no resizing performed, does not work with dataloader since it requires all images of same dimensions'''
train_dataset = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.ToTensor()
                               ]))

'''train_dataset_resize = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.Resize((1024, 1024)),
                                   transforms.Pad((0, 0, 1024, 1024), fill=0),
                                   transforms.ToTensor()
                               ]))'''


'''Dataset with resizing and padding if needed, to be used with dataloader to work in batches'''
train_dataset_eq_size = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   PadToSize(1024, fill=0),
                                   transforms.Resize((1024, 1024), antialias=True),
                                   transforms.ToTensor()
                               ]))

In [24]:
train_dataset_eq_size[0]

{'image': tensor([[[0.6902, 0.6863, 0.6902,  ..., 0.2980, 0.2941, 0.2941],
          [0.6863, 0.6824, 0.6784,  ..., 0.3020, 0.2941, 0.2941],
          [0.7333, 0.7294, 0.7294,  ..., 0.3294, 0.3294, 0.3373],
          ...,
          [0.3647, 0.3725, 0.3686,  ..., 0.5137, 0.4824, 0.5412],
          [0.3451, 0.3569, 0.3608,  ..., 0.5137, 0.5059, 0.4784],
          [0.3255, 0.3451, 0.3608,  ..., 0.5333, 0.5176, 0.4078]],
 
         [[0.7059, 0.7020, 0.7059,  ..., 0.4392, 0.4353, 0.4353],
          [0.6980, 0.6902, 0.6902,  ..., 0.4431, 0.4353, 0.4353],
          [0.7412, 0.7373, 0.7373,  ..., 0.4706, 0.4706, 0.4784],
          ...,
          [0.5020, 0.5098, 0.5059,  ..., 0.5059, 0.4745, 0.5333],
          [0.4824, 0.4941, 0.4980,  ..., 0.5059, 0.4980, 0.4706],
          [0.4627, 0.4824, 0.4980,  ..., 0.5255, 0.5098, 0.4000]],
 
         [[0.7020, 0.6980, 0.7020,  ..., 0.1569, 0.1529, 0.1529],
          [0.6941, 0.6863, 0.6863,  ..., 0.1608, 0.1529, 0.1529],
          [0.7373, 0.7333, 0.73

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset_eq_size, batch_size=1, shuffle=True, num_workers=10, pin_memory=True)

In [4]:
batch_feature_descriptor(train_loader, DEVICE, "sift", "evaluation/sift_descriptors_outputs/")

In [10]:
import h5py

def print_h5_structure(name, obj):
    print(name)

with h5py.File("evaluation/sift_descriptors_outputs/0_1024_sift.h5", "r") as f:
    f.visititems(print_h5_structure)
    print(f["descriptors"][:])
    print(f["keypoints"][:])
    print(f["image_name"].asstr()[:])
    print(f["dataset_name"].asstr()[:])
    print(f["scene_name"].asstr()[:])
    print(f["image"][:])

dataset_name
descriptors
image
image_name
image_path
keypoint_scores
keypoints
scene_name
[[[0.18955745 0.01709182 0.00099998 ... 0.00099998 0.00099998 0.04186624]
  [0.14449462 0.20665532 0.09902858 ... 0.00099998 0.00099998 0.00099998]
  [0.0440099  0.13716894 0.06653672 ... 0.09110915 0.0644239  0.05516933]
  ...
  [0.00099998 0.00099998 0.05110501 ... 0.00099998 0.00099998 0.0312953 ]
  [0.13258754 0.00099999 0.00099999 ... 0.11354119 0.08558489 0.05677059]
  [0.001      0.0164465  0.03677548 ... 0.17170645 0.07887462 0.03289299]]]
[[[838.56226 298.48492]
  [277.97876  99.22626]
  [764.6905  405.86896]
  ...
  [874.76025 730.43463]
  [452.22534 791.18756]
  [705.63025 560.8529 ]]]
['st_pauls_cathedral_15340635_6286567877.png']
['pt_stpeters_stpauls']
['st_pauls_cathedral']
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. .

In [19]:
with h5py.File("evaluation/sift_descriptors_outputs/0_1024_sift.h5", "r") as f:
    print(f["image"][:].squeeze(0))

(3, 1024, 1024)


In [2]:
# pad keypoints and descriptor, save all of the data in one file

# read data from saved files
sift_dir = "evaluation/sift_descriptors_outputs/"

sift_kps = []
sift_desc = []
sift_kps_scores = []
sift_images_name = []
sift_images_path = []
sift_datasets_name = []
sift_scenes_name = []
sift_images = []
for file in os.listdir(sift_dir):
    if file.endswith(".h5"):
        with h5py.File(sift_dir + file, "r") as f:
            sift_kps.append(torch.from_numpy(f["keypoints"][:]).squeeze(0))
            sift_desc.append(torch.from_numpy(f["descriptors"][:]).squeeze(0))
            sift_kps_scores.append(torch.from_numpy(f["keypoint_scores"][:]).squeeze(0))
            sift_images_name += list(f["image_name"].asstr())
            sift_images_path += list(f["image_path"].asstr())
            sift_datasets_name += list(f["dataset_name"].asstr())
            sift_scenes_name += list(f["scene_name"].asstr())
            sift_images.append(f["image"][:].squeeze(0))

In [5]:
np.array(sift_images).shape

(1945, 3, 1024, 1024)

In [6]:
sift_datasets_name

['amy_gardens',
 'imc2024_dioscuri_baalshamin',
 'imc2023_heritage',
 'stairs',
 'pt_brandenburg_british_buckingham',
 'ETs',
 'pt_brandenburg_british_buckingham',
 'imc2024_lizard_pond',
 'pt_sacrecoeur_trevi_tajmahal',
 'fbk_vineyard',
 'imc2023_heritage',
 'pt_brandenburg_british_buckingham',
 'imc2023_heritage',
 'fbk_vineyard',
 'imc2023_heritage',
 'pt_sacrecoeur_trevi_tajmahal',
 'imc2024_lizard_pond',
 'imc2024_lizard_pond',
 'pt_piazzasanmarco_grandplace',
 'pt_sacrecoeur_trevi_tajmahal',
 'pt_brandenburg_british_buckingham',
 'imc2024_dioscuri_baalshamin',
 'imc2024_lizard_pond',
 'pt_stpeters_stpauls',
 'pt_piazzasanmarco_grandplace',
 'fbk_vineyard',
 'imc2024_dioscuri_baalshamin',
 'imc2024_dioscuri_baalshamin',
 'pt_sacrecoeur_trevi_tajmahal',
 'imc2024_dioscuri_baalshamin',
 'stairs',
 'pt_piazzasanmarco_grandplace',
 'imc2024_dioscuri_baalshamin',
 'pt_stpeters_stpauls',
 'fbk_vineyard',
 'pt_brandenburg_british_buckingham',
 'pt_piazzasanmarco_grandplace',
 'imc2023_ha

In [7]:
sift_kps

[tensor([[585.6262, 571.4224],
         [583.6034, 665.5620],
         [583.2990, 501.2842],
         ...,
         [345.5672, 623.4795],
         [666.9412, 831.7335],
         [465.9309, 668.9876]]),
 tensor([[650.3643, 486.7818],
         [190.6458, 789.5334],
         [464.7342, 362.7954],
         ...,
         [919.4657, 719.6495],
         [794.6906, 841.9982],
         [367.8833, 422.8363]]),
 tensor([[587.8468, 380.8715],
         [111.5375, 771.2864],
         [112.5884, 775.0917],
         ...,
         [444.8110, 196.1739],
         [265.0579, 646.9467],
         [413.1328, 770.1012]]),
 tensor([[376.5433,  43.0445],
         [653.4822, 467.9754],
         [653.0467, 692.8986],
         ...,
         [107.7535, 929.8962],
         [778.8806, 250.7323],
         [473.7188, 874.3224]]),
 tensor([[495.1954, 543.9330],
         [495.1978, 212.3945],
         [494.8271, 323.7213],
         ...,
         [546.0725, 420.6625],
         [793.4399, 550.6555],
         [ 14.6972, 544

In [7]:
lengths = [k.size(0) for k in sift_kps]

sift_kps_pad = pad_sequence(sift_kps, batch_first=True, padding_value=0.0)
sift_desc_pad = pad_sequence(sift_desc, batch_first=True, padding_value=0.0)
sift_kps_scores_pad = pad_sequence([s.view(-1).to(dtype=torch.float32) for s in sift_kps_scores],
                              batch_first=True, padding_value=float('-inf'))

B, Nmax = sift_kps_pad.size(0), sift_kps_pad.size(1)
device = sift_kps_pad.device
valid_mask = (torch.arange(Nmax, device=device).unsqueeze(0).expand(B, Nmax) <
      torch.tensor(lengths, device=device).unsqueeze(1))

In [8]:
print(sift_kps_pad.shape)
print(sift_desc_pad.shape)
print(sift_kps_scores_pad.shape)
print(valid_mask.shape)

torch.Size([1945, 1928, 2])
torch.Size([1945, 1928, 128])
torch.Size([1945, 1928])
torch.Size([1945, 1928])


In [13]:
sift_kps_pad

tensor([[[ 838.5623,  298.4849],
         [ 277.9788,   99.2263],
         [ 764.6905,  405.8690],
         ...,
         [   0.0000,    0.0000],
         [   0.0000,    0.0000],
         [   0.0000,    0.0000]],

        [[ 766.0084,  576.1359],
         [ 325.7661,  417.1395],
         [ 704.2563,  434.2031],
         ...,
         [   0.0000,    0.0000],
         [   0.0000,    0.0000],
         [   0.0000,    0.0000]],

        [[ 380.2959,  249.1499],
         [ 643.1902,  699.5665],
         [1020.9064,  464.2120],
         ...,
         [   0.0000,    0.0000],
         [   0.0000,    0.0000],
         [   0.0000,    0.0000]],

        ...,

        [[ 549.8777,  829.4081],
         [ 651.8903,  654.4124],
         [ 549.8065,  269.6503],
         ...,
         [   0.0000,    0.0000],
         [   0.0000,    0.0000],
         [   0.0000,    0.0000]],

        [[   2.6499,  574.6684],
         [ 236.8549,  438.6265],
         [ 241.7405,  578.4001],
         ...,
         [   0.00

In [9]:
# save new data in one file

filename = os.path.join(sift_dir + "merged.h5")

with h5py.File(filename, "w") as f:
    dt = h5py.string_dtype(encoding="utf-8")
    f.create_dataset('keypoints', data=sift_kps_pad.cpu().detach().numpy())
    f.create_dataset('keypoint_scores', data=sift_kps_scores_pad.cpu().detach().numpy())
    f.create_dataset('descriptors', data=sift_desc_pad.cpu().detach().numpy())
    f.create_dataset('keypoints_mask', data=valid_mask.cpu().detach().numpy())
    f.create_dataset('image_name', data=sift_images_name, dtype=dt)
    f.create_dataset('dataset_name', data=sift_datasets_name, dtype=dt)
    f.create_dataset('scene_name', data=sift_scenes_name, dtype=dt)
    f.create_dataset('image_path', data=sift_images_path, dtype=dt)
    f.create_dataset('image', data=np.array(sift_images))

In [14]:
with h5py.File("evaluation/sift_descriptors_outputs/merged.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key].shape
        print(f"{key}: {num_rows} rows")

    kps_merged = np.array(f["keypoints"][:])
    print(f["keypoints"][:])
    # print(f["descriptors"][:])
    # print(f["image_name"].asstr()[:])
    # print(f["image"][:][0])

dataset_name: (1945,) rows
descriptors: (1945, 1928, 128) rows
image: (1945, 3, 1024, 1024) rows
image_name: (1945,) rows
image_path: (1945,) rows
keypoint_scores: (1945, 1928) rows
keypoints: (1945, 1928, 2) rows
keypoints_mask: (1945, 1928) rows
scene_name: (1945,) rows
[[[ 838.56226    298.48492  ]
  [ 277.97876     99.22626  ]
  [ 764.6905     405.86896  ]
  ...
  [   0.           0.       ]
  [   0.           0.       ]
  [   0.           0.       ]]

 [[ 766.0084     576.1359   ]
  [ 325.7661     417.13947  ]
  [ 704.2563     434.20312  ]
  ...
  [   0.           0.       ]
  [   0.           0.       ]
  [   0.           0.       ]]

 [[ 380.29587    249.14992  ]
  [ 643.19025    699.56647  ]
  [1020.9064     464.212    ]
  ...
  [   0.           0.       ]
  [   0.           0.       ]
  [   0.           0.       ]]

 ...

 [[ 549.8777     829.40814  ]
  [ 651.89026    654.4124   ]
  [ 549.80646    269.65033  ]
  ...
  [   0.           0.       ]
  [   0.           0.       ]
 

In [42]:
kps_merged

array([[[ 585.6262 ,  571.4224 ],
        [ 583.6034 ,  665.56195],
        [ 583.29895,  501.2842 ],
        ...,
        [   0.     ,    0.     ],
        [   0.     ,    0.     ],
        [   0.     ,    0.     ]],

       [[ 650.3643 ,  486.78177],
        [ 190.64583,  789.53345],
        [ 464.73422,  362.7954 ],
        ...,
        [   0.     ,    0.     ],
        [   0.     ,    0.     ],
        [   0.     ,    0.     ]],

       [[ 587.8468 ,  380.87146],
        [ 111.53747,  771.28644],
        [ 112.5884 ,  775.09174],
        ...,
        [   0.     ,    0.     ],
        [   0.     ,    0.     ],
        [   0.     ,    0.     ]],

       ...,

       [[1021.1425 ,  215.01436],
        [ 679.2336 ,  829.471  ],
        [ 679.66003,  804.4237 ],
        ...,
        [   0.     ,    0.     ],
        [   0.     ,    0.     ],
        [   0.     ,    0.     ]],

       [[ 454.68423,  252.85074],
        [ 576.4648 ,  431.64352],
        [ 575.9941 ,  289.2771 ],
        .

In [46]:
kps_merged[0]

array([[585.6262 , 571.4224 ],
       [583.6034 , 665.56195],
       [583.29895, 501.2842 ],
       ...,
       [  0.     ,   0.     ],
       [  0.     ,   0.     ],
       [  0.     ,   0.     ]], shape=(1928, 2), dtype=float32)

In [45]:
# example: way to remove the padding if later using flann

kps_without_pad = [kps[(kps[:, 0] != 0) & (kps[:, 1] != 0)] for kps in kps_merged]
kps_without_pad

[array([[585.6262 , 571.4224 ],
        [583.6034 , 665.56195],
        [583.29895, 501.2842 ],
        ...,
        [345.56717, 623.4795 ],
        [666.94116, 831.7335 ],
        [465.9309 , 668.9876 ]], shape=(1758, 2), dtype=float32),
 array([[650.3643 , 486.78177],
        [190.64583, 789.53345],
        [464.73422, 362.7954 ],
        ...,
        [919.4657 , 719.64954],
        [794.6906 , 841.9982 ],
        [367.88333, 422.83627]], shape=(1723, 2), dtype=float32),
 array([[587.8468 , 380.87146],
        [111.53747, 771.28644],
        [112.5884 , 775.09174],
        ...,
        [444.81104, 196.17389],
        [265.05792, 646.9467 ],
        [413.13275, 770.1012 ]], shape=(1660, 2), dtype=float32),
 array([[376.5433  ,  43.044514],
        [653.48224 , 467.9754  ],
        [653.0467  , 692.8986  ],
        ...,
        [107.7535  , 929.8962  ],
        [778.8806  , 250.73228 ],
        [473.71875 , 874.32245 ]], shape=(1715, 2), dtype=float32),
 array([[495.19537 , 543.933   ]

In [44]:
kps_without_pad[0]

array([[585.6262 , 571.4224 ],
       [583.6034 , 665.56195],
       [583.29895, 501.2842 ],
       ...,
       [345.56717, 623.4795 ],
       [666.94116, 831.7335 ],
       [465.9309 , 668.9876 ]], shape=(1758, 2), dtype=float32)

## disk

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset_eq_size, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)

batch_feature_descriptor(train_loader, DEVICE, "disk", "evaluation/disk_descriptors_outputs/")

In [5]:
import h5py
import numpy as np
from pathlib import Path
import glob

root = Path("evaluation/disk_descriptors_outputs")
file_gen = root.glob("[0-9]*.h5")

if not file_gen:
    raise ValueError("No .h5 files found in the folder.")

first_file = next(file_gen)
with h5py.File(first_file, "r") as f:
    keys = list(f.keys())


with h5py.File(root / "merged.h5", "w") as fout:

    for key in keys:
        # Determine the dtype and initial shape from first file
        with h5py.File(first_file, "r") as f:
            data = f[key][...]
        fout.create_dataset(key, data=data, maxshape=(None,) + data.shape[1:])

    #iterate over the remaining files
    for filepath in file_gen:  
        with h5py.File(filepath, "r") as f:
            for key in keys:
                old_data = fout[key]
                new_data = f[key][...]
                old_len = old_data.shape[0]
                new_len = new_data.shape[0]
                old_data.resize(old_len + new_len, axis=0)
                old_data[old_len:] = new_data


In [6]:
def print_h5_structure(name, obj):
    print(name)

with h5py.File("evaluation/disk_descriptors_outputs/merged.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key]
        print(f"{key}: {num_rows} rows")

dataset_name: <HDF5 dataset "dataset_name": shape (1945,), type "|O"> rows
descriptors: <HDF5 dataset "descriptors": shape (1945, 2048, 128), type "<f4"> rows
image: <HDF5 dataset "image": shape (1945, 3, 1024, 1024), type "<f4"> rows
image_name: <HDF5 dataset "image_name": shape (1945,), type "|O"> rows
image_path: <HDF5 dataset "image_path": shape (1945,), type "|O"> rows
keypoint_scores: <HDF5 dataset "keypoint_scores": shape (1945, 2048), type "<f4"> rows
keypoints: <HDF5 dataset "keypoints": shape (1945, 2048, 2), type "<f4"> rows
keypoints_mask: <HDF5 dataset "keypoints_mask": shape (1945, 2048), type "<f4"> rows
scene_name: <HDF5 dataset "scene_name": shape (1945,), type "|O"> rows


In [6]:
with h5py.File("evaluation/disk_descriptors_outputs/0_1024_disk.h5", "r") as f:
    print(f["image_name"].asstr()[:])
    # print(f["image"][:])
    print(f["keypoints"][:].shape)

['brandenburg_gate_93110830_1932760362.png' 'peach_0150.png'
 'piazza_san_marco_59702055_3587019674.png'
 'brandenburg_gate_54744886_13665893353.png' 'lizard_00074.png'
 'pond_00644.png' 'church_00093.png' 'sacre_coeur_47788810_3531619701.png']
(8, 2048, 2)
